## Custom Difficulty RPG
- Use Monte Carlo to find out the % chance of winning an enemy
- Tune the % chance of winning to desired difficulty!!


# Coding the Interface

In [66]:
import numpy as np
from IPython.display import clear_output

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [67]:
def PrintInterface():
    print("Own Hp:", ownhp)
    print("Enemy Hp:", enemyhp)
    print()
    print(color.BOLD + "Available Moves:" + color.END)
    for skill in ownskills:
        if ownskills[skill].isavailable():
            print(skill, str(vars(ownskills[skill])))
        
    print()
    print(color.BOLD + "Moves on Cooldown:" + color.END)
    hascooldown = False
    for skill in ownskills:
        if not ownskills[skill].isavailable():
            print(skill, str(vars(ownskills[skill])))  
            hascooldown = True
    if not hascooldown:
        print("None")
    print()

In [80]:
class Skill:
    def __init__(self, dmg, cd, moves_to_use = 0):
        self.dmg = dmg
        self.cd = cd
        self.moves_to_use = moves_to_use
        
    # Returns if the skill can be used
    def isavailable(self):
        return self.moves_to_use == 0
        
    # Use the current skill
    def useskill(self):
        self.moves_to_use = self.cd + 1
        
    # delays a turn 
    def delayturn(self):
        self.moves_to_use = max(0, self.moves_to_use-1)
        
    # resets the cd
    def reset(self):
        self.moves_to_use = 0

In [91]:
# Returns the desired enemy hp to get a scaled %hp difficulty
# Also returns as optional parameter, the sequence of moves
def GetCustomDifficulty(ownhp, enemyattack, ownskills, difficulty = 0.9, numgames = 10):
    besthp = 0
    bestsequence = []
    for i in range(numgames):
        currenthp = 0
        currentsequence = []
        newownhp = ownhp
        for skill in ownskills:
            ownskills[skill].reset()
            
        # Gameplay Interface
        while newownhp > 0:
            # Delay a turn for all moves
            for skill in ownskills:
                ownskills[skill].delayturn()

            # Select a random skill
            skill = np.random.choice(list(ownskills.keys()))
            while not (skill.lower() in ownskills and ownskills[skill].isavailable()):
                skill = np.random.choice(list(ownskills.keys()))
            currentsequence.append(skill)

            # Enemy gets damaged
            currenthp += ownskills[skill].dmg
            ownskills[skill].useskill()

            # You get damaged
            newownhp -= enemyattack
            
        if currenthp > besthp:
            besthp = currenthp
            bestsequence = currentsequence
        
    return int(besthp * difficulty), bestsequence

In [105]:
ownhp = 100
enemyhp = 100
enemyattack = 10
ownskills = {"attack": Skill(10, 0), 
             "fire ball": Skill(20, 1), 
             "ice bolt": Skill(50, 2),
            "lightning": Skill(1000, 5)}

# Use Monte Carlo Method to estimate difficulty
enemyhp, bestseq = GetCustomDifficulty(ownhp, enemyattack, ownskills, difficulty = 0.9, numgames = 100)
print(bestseq)

for skill in ownskills:
    ownskills[skill].reset()

# Gameplay Interface
while ownhp > 0 and enemyhp > 0:
    # Delay a turn for all moves
    for skill in ownskills:
        ownskills[skill].delayturn()
        
    # Print Interface
    PrintInterface()
    skill = input("Key in a move")
    # Keep repeating the move until it is a valid move
    while not (skill.lower() in ownskills and ownskills[skill].isavailable()):
        skill = input("Key in a move: ")
        
    clear_output(wait=True)
    
    # Enemy gets damaged
    enemyhp -= ownskills[skill].dmg
    ownskills[skill].useskill()
    
    print(color.BOLD + "Turn Summary:" + color.END)
    print(f"You hit enemy with {skill} for {ownskills[skill].dmg} damage.")
    
    
    # You get damaged
    if enemyhp > 0:
        ownhp -= enemyattack
        print(f"The enemy hits you for {enemyattack} damage.")
        print()
    
if ownhp > 0:
    print("You win!")
else:
    print("You lose!")

Turn Summary:
You hit enemy with attack for 10 damage.
You win!
